In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch
import h5py
import random

from totalface_cpu.model_zoo.model_common import load_onnx

In [ ]:
def normalization(rgb_img,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    MEAN = 255 * np.array(mean_list)
    STD = 255 * np.array(std_list)
    rgb_img = rgb_img.transpose(-1, 0, 1)
    norm_img = (rgb_img - MEAN[:, None, None]) / STD[:, None, None]
    
    return norm_img

In [ ]:
def read_image(path,bbox,mean,std,resize=128,prefix='',ori_return=False):
    img = cv2.imread(prefix+path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    cropped = img[bbox[1]:bbox[3],bbox[0]:bbox[2]]

    input_img = cv2.resize(cropped,(resize,resize),interpolation=cv2.INTER_CUBIC)
    input_img = normalization(input_img,mean,std)
    input_img = np.transpose(input_img,(1,2,0))
    
    if ori_return:
        return input_img, img
    else:
        return input_img

In [ ]:
# model load 
model_path = "./pretrained/anti-spoof-mn3.onnx"
model = load_onnx.Onnx_session(model_path,input_mean=0.0, input_std=1.0,output_sort=True)
# input (1,3,128,128)
# output (1,2)

# result label
pred_dict={0:'real',1:'fake'}

In [ ]:
# image param
mean=[0.5931, 0.4690, 0.4229]
std=[0.2471, 0.2214, 0.2157]
resize=128

In [ ]:
# dataset
h5_path = "FaceInTheWild/results_scrfd_10g/wild_test.h5"
hf = h5py.File(h5_path,'r')

paths = hf['path']
bboxs = hf['bboxs']
labels = hf['labels']
indexs = range(len(paths))

prefix = "/data/"

In [ ]:
choice_idx = random.choice(indexs)

In [ ]:
path = paths[choice_idx]
bbox = bboxs[choice_idx]
label = labels[choice_idx]

In [ ]:
# input
input_img , img = read_image(path,bbox,mean,std,resize=resize,prefix=prefix,ori_return=True)

In [ ]:
# run
out = model(input_img)[0][0] # prob (real,fake)
pred_idx = np.argmax(out)

In [ ]:
print(out) # real, fake
print("gt: {} / pred: {}".format(label,pred_dict[pred_idx]))

In [ ]:
plt.imshow(img)